In [1]:
import pandas as pd 
import numpy as np
import random
import datetime
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt 
import utilities as utils
import fractionalDiff as fd
import pickle
import scipy.stats as sps
from sys import exit

In [2]:
inputs = pd.read_excel('data/candles.xlsx')
logicOutputs = pd.read_excel('data/bitcoin.xlsx')
tickers = ['indicator']
inputVariables = inputs.columns[1:]

# maWindow = 11


# inputs['weekDay'] = inputs['date'].apply(lambda x: datetime.datetime.weekday(x))
# inputs['yearWeek'] = pd.DatetimeIndex(inputs['date']).isocalendar().week[0] * 10 + inputs['date'].apply(lambda x: datetime.datetime.weekday(x))

# for rollWndw in range(4, maWindow):
#     inputs[[f'mah{rollWndw}', f'mal{rollWndw}', f'mac{rollWndw}', f'mav{rollWndw}', f'maubb{rollWndw}', f'malbb{rollWndw}']] = inputs[['high', 'low', 'close', 'volume', 'UpperBB', 'LowerBB']].rolling(maWindow).mean()

# inputs.dropna(inplace = True)
# inputs.reset_index(inplace = True)
# logicOutputs = logicOutputs.iloc[(maWindow-1):, :]
# inputs.shape, logicOutputs.shape

In [3]:
inputs

,date,open,high,low,close,volume,ma_04,ma_06,ma_12,ma_16,ma_18,ma_20,ma_22,ma_24,ma_30,ma_48
0,2022-06-02 02:00:00,29805.84,29817.89,29594.55,29680.00,2826.62933,29706.4800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-06-02 03:00:00,29680.01,29888.02,29640.18,29811.89,1742.44118,29761.2200,29712.316667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-02 04:00:00,29811.90,29903.74,29778.12,29826.16,1292.33107,29780.9700,29743.995000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-02 05:00:00,29826.16,29856.93,29703.05,29730.12,1690.59886,29762.0425,29766.860000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-02 06:00:00,29730.12,29824.11,29650.93,29817.24,1223.11546,29796.3525,29778.540000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,2022-06-14 05:00:00,21548.74,22300.00,21327.36,22127.91,11194.43061,21572.7900,21894.411667,22628.232500,22809.387500,22926.125556,23045.2845,23230.700909,23404.160417,23940.334000,25308.005000
292,2022-06-14 06:00:00,22127.92,22466.88,21763.71,21977.83,10250.83244,21627.3700,21792.733333,22481.928333,22702.795000,22814.303333,22935.0460,23077.050455,23261.987083,23773.080333,25194.279792
293,2022-06-14 07:00:00,21977.83,23362.88,21918.66,22747.24,15621.63646,22100.4300,21836.038333,22416.004167,22646.251250,22759.737222,22869.7665,22983.216364,23158.353750,23645.504000,25100.039167
294,2022-06-14 08:00:00,22747.25,22909.50,22323.78,22633.82,8974.26356,22371.7000,21981.756667,22354.238333,22635.576250,22701.432222,22801.9260,22912.817273,23044.840417,23523.059333,24998.956458


In [4]:
def TrendFollowOutput(data, date):
    prediction = 2.*((data.loc[data.date == date, 'close'].values - data.loc[data.date == date, 'open'].values)[0] >= 0) - 1.
    return prediction

In [8]:
a=fd.FractionallyDifferentiateInputs(inputs, inputVariables, order = 0.8, thresholdVal = 200, cutoff=False)

In [11]:
scoresByOrderLst = []
mlModels = []
window_0 = 3
deltaWindow = 7
cutoff = 1e-2
orders = np.linspace(0.1, 1.0, 10)

windows = range(window_0, window_0 + deltaWindow + 1)
lags = 50#np.array([fd.findCutoff(order, cutoff, 1) for order in orders]).max()
#windows = [4, 9, 10, 12, 14, 16, 18, 20, 22, 24]

firstOrderDifferences = fd.FractionallyDifferentiateInputs(inputs, inputVariables, order = 1.0, thresholdVal = lags, cutoff=False)[fd.findCutoff(1.0, cutoff, 1):].reset_index()

transformedInputsLst = []
for order in orders:
    print(f'Calculating differencing orders: {order}')
    transformedInputs = fd.FractionallyDifferentiateInputs(inputs, inputVariables, order = order, thresholdVal = lags, cutoff=False)[lags:].reset_index(drop=True)
    #transformedInputs.drop(columns='index', inplace = True)
    transformedInputsLst.append(transformedInputs)

transformedInputsAllOrders = pd.concat(transformedInputsLst, axis = 1)
transformedInputsAllOrders.drop(columns = 'date', inplace = True)



for orderNumber, order in enumerate(orders):
    print(f'Differencing order: {order}')
    transformedInputs = transformedInputsLst[orderNumber]
    recapWithOrdersLst = []
    simRecapDatesLst = []

    for windowNumber, window in enumerate(windows):
        print(f'Recalibration window: {window}')
        trainScores = []
        testScores = []
        testScoresAlternativeMdl = []
        tickersLst = []
        trainOrderLst = []
        simulationDatesLst = []
        simulationDates = transformedInputs.date.unique()[(window):(transformedInputs.date.shape[0] - 1)]
        strategyReturnLst = []
        alternativeMdlStrategyReturnLst = []
        assetReturnByDateLst = []

        for simulationDate in simulationDates:
            
            startDateIdx = transformedInputs.index[transformedInputs.date == simulationDate][0] - window
            stopDateIdx = transformedInputs.index[transformedInputs.date == simulationDate][0] + 1
            
            startDate = transformedInputs.date[startDateIdx]
            testDate = transformedInputs.date[stopDateIdx - 1]
            stopDate = transformedInputs.date[stopDateIdx]
            assetReturnLst = []

            for ticker in tickers:
                
                pt0 = inputs.loc[inputs.date == stopDate, 'open']
                filteredReturn = inputs.loc[inputs.date == stopDate, 'close'] - inputs.loc[inputs.date == stopDate, 'open'] 
                inputsToScale = utils.FilterInputs(transformedInputs, startDate, stopDate)
                x, y = utils.SelectAndScale(inputsToScale, logicOutputs[lags:], startDate, stopDate, ticker, alreadyScaled = False)
                x = np.nan_to_num(x)
                xTrain, xTest, yTrain, yTest = utils.TestTrainSplit(x, y, window)             

                if len(np.unique(yTrain)) < 2:
                    continue
                
                lrm = skl_lm.LogisticRegression(random_state=0, verbose = 0).fit(xTrain, yTrain)#beta =...

                if simulationDate == max(simulationDates):
                    mlModels.append({'order':order, 'calibrationWindow': window, 'model': lrm})

                trainScore = lrm.score(xTrain, yTrain)
                testScore = lrm.score(xTest, yTest)
                yPred = lrm.predict(xTest)

                yPredAlternativeMdl = TrendFollowOutput(inputs, simulationDate)
                trainScore = 0
                testScoreAlternativeMdl = 1. * (yTest == yPredAlternativeMdl)
                strategyReturn = yPred[0] * filteredReturn.values[0] / pt0.values[0] - 0.001
                alternativeMdlStrategyReturn = yPredAlternativeMdl * filteredReturn.values[0] / pt0.values[0] - 0.001 

                trainScores.append(trainScore)
                testScores.append(testScore)
                testScoresAlternativeMdl.append(testScoreAlternativeMdl[0])
                simulationDatesLst.append(simulationDate)
                tickersLst.append(ticker)
                trainOrderLst.append(window)
                strategyReturnLst.append(strategyReturn)
                alternativeMdlStrategyReturnLst.append(alternativeMdlStrategyReturn)
                assetReturnLst.append(filteredReturn.values[0]  / pt0.values[0] - 0.001)
            assetReturnByDateLst.append(assetReturnLst)
            assetReturnByDateFlatLst = [item for sublist in assetReturnByDateLst for item in sublist]

        simRecapDf = pd.DataFrame(list(zip(trainOrderLst, simulationDatesLst, tickersLst, trainScores, testScores, testScoresAlternativeMdl, strategyReturnLst, alternativeMdlStrategyReturnLst, assetReturnByDateFlatLst)), columns = ['window', 'simulationDate', 'ticker', 'trainScore', 'testScore', 'alternativeMdlTestScore', 'strategyReturn', 'alternativeMdlStrategyReturn', 'assetReturn'])
        simRecapDatesLst.append(simRecapDf)

    simRecapWithWindowsDf = pd.concat(simRecapDatesLst)
    simRecapWithWindowsDf['order'] = order
    scoresByOrderLst.append(simRecapWithWindowsDf)

overallScores = pd.concat(scoresByOrderLst)
#overallScores.loc[overallScores.ticker.str.contains('JPY'), 'strategyReturn'] = overallScores.loc[overallScores.ticker.str.contains('JPY'), 'strategyReturn']/100.

Calculating differencing orders: 0.1
Calculating differencing orders: 0.2
Calculating differencing orders: 0.30000000000000004
Calculating differencing orders: 0.4
Calculating differencing orders: 0.5
Calculating differencing orders: 0.6
Calculating differencing orders: 0.7000000000000001
Calculating differencing orders: 0.8
Calculating differencing orders: 0.9
Calculating differencing orders: 1.0
Differencing order: 0.1
Recalibration window: 3
Recalibration window: 4
Recalibration window: 5
Recalibration window: 6
Recalibration window: 7
Recalibration window: 8
Recalibration window: 9
Recalibration window: 10
Differencing order: 0.2
Recalibration window: 3
Recalibration window: 4
Recalibration window: 5
Recalibration window: 6
Recalibration window: 7
Recalibration window: 8
Recalibration window: 9
Recalibration window: 10
Differencing order: 0.30000000000000004
Recalibration window: 3
Recalibration window: 4
Recalibration window: 5
Recalibration window: 6
Recalibration window: 7
Recal

In [14]:
inputs.shape

(296, 16)

In [12]:
#alt model: fractional trend following
aggScores = overallScores.groupby(['order', 'window']).agg({'testScore':[np.size, np.mean, sps.skew, sps.kurtosis, np.std], \
    'strategyReturn': [np.sum], 'alternativeMdlTestScore':[np.mean, sps.skew, sps.kurtosis, np.std], 'alternativeMdlStrategyReturn': [np.sum], 'assetReturn':[np.sum]}).droplevel(level=[0], axis = 1).reset_index()
aggScores.columns = ['order', 'window', 'count', 'testScore', 'skew', 'kurt', 'testStd', 
    'strategyReturn', 'alternativeMdlTestScore', 'amSkew', 'amKurt', 'amtestStd', 'alternativeMdlStrategyReturn', 'assetReturn']

aggScores.sort_values(by = ['strategyReturn'], ascending = False)

,order,window,count,testScore,skew,kurt,testStd,strategyReturn,alternativeMdlTestScore,amSkew,amKurt,amtestStd,alternativeMdlStrategyReturn,assetReturn
55,0.7,10,235,0.565957,-0.266156,-1.929161,0.496688,0.106755,0.463830,0.145061,-1.978957,0.499754,-0.019393,-0.500370
54,0.7,9,235,0.553191,-0.213980,-1.954212,0.498224,0.084617,0.468085,0.127920,-1.983636,0.500045,-0.007502,-0.512261
18,0.3,5,225,0.533333,-0.133631,-1.982143,0.500000,0.075721,0.457778,0.169494,-1.971272,0.499325,-0.031636,-0.450936
6,0.1,9,235,0.553191,-0.213980,-1.954212,0.498224,0.062467,0.468085,0.127920,-1.983636,0.500045,-0.007502,-0.512261
10,0.2,5,225,0.515556,-0.062252,-1.996125,0.500872,0.053739,0.457778,0.169494,-1.971272,0.499325,-0.031636,-0.450936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.9,4,216,0.509259,-0.037043,-1.998628,0.501076,-0.143005,0.449074,0.204769,-1.958070,0.498555,-0.093395,-0.376134
0,0.1,3,193,0.445596,0.218916,-1.952076,0.498324,-0.143258,0.450777,0.197852,-1.960854,0.498865,-0.133868,-0.347257
16,0.3,3,193,0.466321,0.135022,-1.981769,0.500162,-0.144734,0.450777,0.197852,-1.960854,0.498865,-0.133868,-0.347257
24,0.4,3,193,0.461140,0.155912,-1.975691,0.499784,-0.162686,0.450777,0.197852,-1.960854,0.498865,-0.133868,-0.347257


In [10]:
#alt model: fractional trend following
aggScores = overallScores.groupby(['order', 'window']).agg({'testScore':[np.size, np.mean, sps.skew, sps.kurtosis, np.std], \
    'strategyReturn': [np.sum], 'alternativeMdlTestScore':[np.mean, sps.skew, sps.kurtosis, np.std], 'alternativeMdlStrategyReturn': [np.sum], 'assetReturn':[np.sum]}).droplevel(level=[0], axis = 1).reset_index()
aggScores.columns = ['order', 'window', 'count', 'testScore', 'skew', 'kurt', 'testStd', 
    'strategyReturn', 'alternativeMdlTestScore', 'amSkew', 'amKurt', 'amtestStd', 'alternativeMdlStrategyReturn', 'assetReturn']

aggScores.sort_values(by = ['strategyReturn'], ascending = False)

,order,window,count,testScore,skew,kurt,testStd,strategyReturn,alternativeMdlTestScore,amSkew,amKurt,amtestStd,alternativeMdlStrategyReturn,assetReturn
71,0.9,10,85,0.611765,-0.458664,-1.789627,0.490241,0.133710,0.529412,-0.117851,-1.986111,0.502096,0.064405,-0.337497
55,0.7,10,85,0.623529,-0.509925,-1.739976,0.487376,0.125433,0.529412,-0.117851,-1.986111,0.502096,0.064405,-0.337497
63,0.8,10,85,0.623529,-0.509925,-1.739976,0.487376,0.110716,0.529412,-0.117851,-1.986111,0.502096,0.064405,-0.337497
6,0.1,9,85,0.635294,-0.562147,-1.683990,0.484204,0.105489,0.529412,-0.117851,-1.986111,0.502096,0.069456,-0.346547
54,0.7,9,85,0.611765,-0.458664,-1.789627,0.490241,0.095614,0.529412,-0.117851,-1.986111,0.502096,0.069456,-0.346547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0.8,3,70,0.542857,-0.172062,-1.970395,0.501757,-0.138892,0.485714,0.057166,-1.996732,0.503405,-0.045980,-0.225852
40,0.6,3,70,0.485714,0.057166,-1.996732,0.503405,-0.140597,0.485714,0.057166,-1.996732,0.503405,-0.045980,-0.225852
73,1.0,4,78,0.538462,-0.154303,-1.976190,0.501745,-0.142367,0.474359,0.102699,-1.989453,0.502574,-0.027392,-0.243173
57,0.8,4,78,0.525641,-0.102699,-1.989453,0.502574,-0.148020,0.474359,0.102699,-1.989453,0.502574,-0.027392,-0.243173


In [13]:
#alt model: fractional trend following
aggScores = overallScores.groupby(['order', 'window']).agg({'testScore':[np.size, np.mean, sps.skew, sps.kurtosis, np.std], \
    'strategyReturn': [np.sum], 'alternativeMdlTestScore':[np.mean, sps.skew, sps.kurtosis, np.std], 'alternativeMdlStrategyReturn': [np.sum], 'assetReturn':[np.sum]}).droplevel(level=[0], axis = 1).reset_index()
aggScores.columns = ['order', 'window', 'count', 'testScore', 'skew', 'kurt', 'testStd', 
    'strategyReturn', 'alternativeMdlTestScore', 'amSkew', 'amKurt', 'amtestStd', 'alternativeMdlStrategyReturn', 'assetReturn']

aggScores.sort_values(by = ['strategyReturn'], ascending = False)

,order,window,count,testScore,skew,kurt,testStd,strategyReturn,alternativeMdlTestScore,amSkew,amKurt,amtestStd,alternativeMdlStrategyReturn,assetReturn
63,0.8,10,85,0.623529,-0.509925,-1.739976,0.487376,0.129601,0.529412,-0.117851,-1.986111,0.502096,0.064405,-0.337497
79,1.0,10,85,0.600000,-0.408248,-1.833333,0.492805,0.089974,0.529412,-0.117851,-1.986111,0.502096,0.064405,-0.337497
6,0.1,9,85,0.600000,-0.408248,-1.833333,0.492805,0.072357,0.529412,-0.117851,-1.986111,0.502096,0.069456,-0.346547
71,0.9,10,85,0.600000,-0.408248,-1.833333,0.492805,0.068337,0.529412,-0.117851,-1.986111,0.502096,0.064405,-0.337497
62,0.8,9,85,0.600000,-0.408248,-1.833333,0.492805,0.062669,0.529412,-0.117851,-1.986111,0.502096,0.069456,-0.346547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0.8,3,70,0.542857,-0.172062,-1.970395,0.501757,-0.153163,0.485714,0.057166,-1.996732,0.503405,-0.045980,-0.225852
48,0.7,3,70,0.528571,-0.114473,-1.986896,0.502787,-0.156207,0.485714,0.057166,-1.996732,0.503405,-0.045980,-0.225852
57,0.8,4,78,0.500000,0.000000,-2.000000,0.503236,-0.169655,0.474359,0.102699,-1.989453,0.502574,-0.027392,-0.243173
8,0.2,3,70,0.414286,0.348009,-1.878890,0.496155,-0.176247,0.485714,0.057166,-1.996732,0.503405,-0.045980,-0.225852


In [11]:
#alt model: fractional trend following
aggScores = overallScores.groupby(['order', 'window']).agg({'testScore':[np.size, np.mean, sps.skew, sps.kurtosis, np.std], \
    'strategyReturn': [np.sum], 'alternativeMdlTestScore':[np.mean, sps.skew, sps.kurtosis, np.std], 'alternativeMdlStrategyReturn': [np.sum], 'assetReturn':[np.sum]}).droplevel(level=[0], axis = 1).reset_index()
aggScores.columns = ['order', 'window', 'count', 'testScore', 'skew', 'kurt', 'testStd', 
    'strategyReturn', 'alternativeMdlTestScore', 'amSkew', 'amKurt', 'amtestStd', 'alternativeMdlStrategyReturn', 'assetReturn']

aggScores.sort_values(by = ['strategyReturn'], ascending = False)

,order,window,count,testScore,skew,kurt,testStd,strategyReturn,alternativeMdlTestScore,amSkew,amKurt,amtestStd,alternativeMdlStrategyReturn,assetReturn
62,0.8,9,235,0.565957,-0.266156,-1.929161,0.496688,0.072029,0.468085,0.127920,-1.983636,0.500045,-0.007502,-0.512261
2,0.1,5,225,0.506667,-0.026669,-1.999289,0.501070,0.067487,0.457778,0.169494,-1.971272,0.499325,-0.031636,-0.450936
10,0.2,5,225,0.511111,-0.044455,-1.998024,0.500991,0.046503,0.457778,0.169494,-1.971272,0.499325,-0.031636,-0.450936
6,0.1,9,235,0.548936,-0.196689,-1.961313,0.498662,0.040466,0.468085,0.127920,-1.983636,0.500045,-0.007502,-0.512261
18,0.3,5,225,0.528889,-0.115749,-1.986602,0.500278,0.039988,0.457778,0.169494,-1.971272,0.499325,-0.031636,-0.450936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,0.9,7,233,0.502146,-0.008584,-1.999926,0.501072,-0.141793,0.459227,0.163635,-1.973224,0.499408,-0.059690,-0.459315
57,0.8,4,216,0.490741,0.037043,-1.998628,0.501076,-0.156587,0.449074,0.204769,-1.958070,0.498555,-0.093395,-0.376134
65,0.9,4,216,0.495370,0.018519,-1.999657,0.501140,-0.178219,0.449074,0.204769,-1.958070,0.498555,-0.093395,-0.376134
16,0.3,3,193,0.466321,0.135022,-1.981769,0.500162,-0.180315,0.450777,0.197852,-1.960854,0.498865,-0.133868,-0.347257


In [7]:
#alt model: fractional trend following
aggScores = overallScores.groupby(['order', 'window']).agg({'testScore':[np.size, np.mean, sps.skew, sps.kurtosis, np.std], \
    'strategyReturn': [np.sum], 'alternativeMdlTestScore':[np.mean, sps.skew, sps.kurtosis, np.std], 'alternativeMdlStrategyReturn': [np.sum], 'assetReturn':[np.sum]}).droplevel(level=[0], axis = 1).reset_index()
aggScores.columns = ['order', 'window', 'count', 'testScore', 'skew', 'kurt', 'testStd', 
    'strategyReturn', 'alternativeMdlTestScore', 'amSkew', 'amKurt', 'amtestStd', 'alternativeMdlStrategyReturn', 'assetReturn']

aggScores.sort_values(by = ['strategyReturn'], ascending = False)

,order,window,count,testScore,skew,kurt,testStd,strategyReturn,alternativeMdlTestScore,amSkew,amKurt,amtestStd,alternativeMdlStrategyReturn,assetReturn
18,0.3,5,225,0.524444,-0.097895,-1.990417,0.500516,0.074271,0.457778,0.169494,-1.971272,0.499325,-0.031636,-0.450936
2,0.1,5,225,0.493333,0.026669,-1.999289,0.501070,0.059229,0.457778,0.169494,-1.971272,0.499325,-0.031636,-0.450936
10,0.2,5,225,0.515556,-0.062252,-1.996125,0.500872,0.046579,0.457778,0.169494,-1.971272,0.499325,-0.031636,-0.450936
62,0.8,9,235,0.544681,-0.179441,-1.967801,0.499063,0.039416,0.468085,0.127920,-1.983636,0.500045,-0.007502,-0.512261
6,0.1,9,235,0.548936,-0.196689,-1.961313,0.498662,0.038163,0.468085,0.127920,-1.983636,0.500045,-0.007502,-0.512261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1.0,6,231,0.489177,0.043300,-1.998125,0.500968,-0.174358,0.454545,0.182574,-1.966667,0.499011,-0.065420,-0.442274
16,0.3,3,193,0.450777,0.197852,-1.960854,0.498865,-0.188996,0.450777,0.197852,-1.960854,0.498865,-0.133868,-0.347257
8,0.2,3,193,0.455959,0.176853,-1.968723,0.499352,-0.200983,0.450777,0.197852,-1.960854,0.498865,-0.133868,-0.347257
73,1.0,4,216,0.490741,0.037043,-1.998628,0.501076,-0.206282,0.449074,0.204769,-1.958070,0.498555,-0.093395,-0.376134


In [24]:
overallScores.to_csv('out_data/overallScores_1_hour_BearishBTC.csv', index = False)